In [1]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


#df = pd.read_csv('train.csv')

In [2]:
test_data = pd.read_csv('test3_all.csv')

# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)



print(len(vectorizer.vocabulary_))


# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv", header=None)
crp.to_csv("corpus1.csv", encoding="utf-8", header=['words','id'], index=None)

#LOAD DATA
crp = pd.read_csv("corpus1.csv")

#CLEAN DATA
crp['words'] = crp['words'].str.replace("\d+", '')
crp['words'] = crp['words'].str.replace("sec", '')
crp['words'] = crp['words'].str.replace("nabkv", '')
# delete values less then 3
crp =crp[crp['words'].str.len() > 3]
# delete geolat geolon
crp['words'] = crp['words'].str.replace("geolat", '')
crp['words'] = crp['words'].str.replace("geolon", '')
crp['words'] = crp['words'].str.replace("geotagged", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
crp['words'].replace('', np.nan, inplace=True)
crp = crp.dropna(subset=['words'])

crp = crp[crp['words'].str.len() > 3]

# save data
crp.to_csv("corpus.csv", encoding="utf-8",index=None)

# load data again 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']



63463


In [ ]:
# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

vocab = tfidf.vocabulary_

# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean1.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

nbrs = NearestNeighbors(n_neighbors = 1, algorithm='ball_tree')
nbrs.fit(train_c_matrix)

test_data = pd.read_csv('test3_all.csv')
# delete all data with empty tags from test data
#test_data.dropna(subset=['photoTags'], inplace=True)
test_data['photoTags'].replace( np.nan,'', inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)

predicted_lt = []
predicted_lng = []

for i in range(0,len(test_data_)):
    distances, indices = nbrs.kneighbors(test_c_matrix[i])

    ind = indices[0][0]

    #print('ind',ind)
    #print(distances)

    lt =train_data.loc[ind,'latitude']
    predicted_lt.append(lt)
    lng = train_data.loc[ind,'longitude']
    predicted_lng.append(lng)
    
    #df_lt = pd.DataFrame([lt], columns=['lt'])
    #df_ln = pd.DataFrame([lng], columns=['ln'])
    
    #data_lt = data_lt.append(df_lt)
    #data_ln = data_ln.append(df_ln)
    #print('test:', test_data.loc[i,'photoTags'])
    #print('train:', train_data.loc[ind,'photoTags'])
    #print('-----------------------------')

C:\Apps\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [ ]:
test_data['lt_predicted'] = predicted_lt
test_data['lng_predicted'] = predicted_lng

test_data.to_csv("predicted_2.csv", encoding="utf-8")

In [12]:
data_predicted = pd.read_csv('predicted_2.csv')

# let's calculate the error and save it in a new column 
lt_error = (data_predicted['lt_predicted'] - data_predicted['latitude'])**(2)
ln_error = (data_predicted['lng_predicted'] - data_predicted['longitude'])**(2)
er = (lt_error + ln_error)**(0.5)
data_predicted['error'] = er

# we will see radius of 0.001 km, 0.01 km, 1 km, 10 km , 100 km , 1000 km, 10000 km, 40000 km 

data_predicted.to_csv("predicted_2_error.csv", encoding="utf-8")